In [128]:
import os
import re
import numpy as np
import pandas as pd
import nltk
import plotly as py
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_distplot
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import ProbPlot
py.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [12]:
#read in the data
nyrv = pd.read_csv('./Datasets/2018_NY_March_reviews.csv')



In [13]:
nyrv.shape


(896208, 6)

In [14]:
nyrv.head()




,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2515,198,2008-10-13,2603,Jenny,Stephanie was a wonderful host! Her apartment ...
1,2515,859,2009-03-08,8455,Roland,Such a wonderful place and very close to the m...
2,2515,1083,2009-03-25,9759,Cem,I just got back from a trip to NYC during whic...
3,2515,1107,2009-03-27,9193,Holly,Stephanie's offered all the most important thi...
4,2515,2175,2009-05-09,7048,Alessandra,"Stephanie was really nice, ftiendly and helpfu..."


In [15]:
nyrv.isnull().sum()

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      0
comments         855
dtype: int64

In [16]:
#dropping the rows that don't have comments
nyrv = nyrv[nyrv.comments.notnull()]

In [17]:
nyrv.shape

(895353, 6)

In [18]:
nyrv.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [19]:
nyrv

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2515,198,2008-10-13,2603,Jenny,Stephanie was a wonderful host! Her apartment ...
1,2515,859,2009-03-08,8455,Roland,Such a wonderful place and very close to the m...
2,2515,1083,2009-03-25,9759,Cem,I just got back from a trip to NYC during whic...
3,2515,1107,2009-03-27,9193,Holly,Stephanie's offered all the most important thi...
4,2515,2175,2009-05-09,7048,Alessandra,"Stephanie was really nice, ftiendly and helpfu..."
5,2515,2889,2009-05-20,12521,Michael,I was pleasantly surprised with my whole stay ...
6,2515,3213,2009-05-27,5889,Becca,Stephanie's apt was great. Very convenient to ...
7,2515,3313,2009-05-30,15342,Sarah,my husband and i had such a great time staying...
8,2515,7144,2009-08-10,26899,Maria Luisa,"When I first arrived at Stephanie's, she was v..."
9,2515,7350,2009-08-13,26718,Greg,awesome couldn't have been better.


In [24]:
nydf = pd.read_csv('./Datasets/2018_NY_March_Cleaned_listings.csv')

/Users/wenchang_qian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning:

Columns (0,15,27,38,43,44,49,51,55,56,61,62,65) have mixed types. Specify dtype option on import or set low_memory=False.



In [25]:
nylisting = nydf.copy()

In [26]:
nylisting.description

0        Step into our artistic spacious apartment and ...
1        Renovated apt home in elevator building. Spaci...
2        Find your romantic getaway to this beautiful, ...
3        This is a spacious, clean, furnished master be...
4        WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...
5        Urban retreat: enjoy 500 s.f. floor in 1899 br...
6        In the heart of it all!  Message me about avai...
7        I'm in a perfect location of Manhattan, being ...
8        HELLO EVERYONE AND THANKS FOR VISITING BLISS A...
9        We welcome you to stay in our lovely 2 br +bab...
10       Quiet, very small one bedroom apartment in Mur...
11       You will use one large, furnished, private roo...
12       Our best guests are seeking a safe, clean, spa...
13       Live like a New Yorker instead of a stuffy, ti...
14       Beautiful 1br on the Upper West Side. Apartmen...
15       BOOKING FOR THREE OR FOUR WEEKS DURING THE MON...
16       The apartment is minutes from the Broadway the.

In [36]:
nylisting['totaldes'] = nylisting.description + ' ' + nylisting.amenities

In [37]:
nylisting.description[0]

"Step into our artistic spacious apartment and enjoy your artistic Guest room with original artwork from NY artists. Shared with my little family however we often out and you won't see us much. across the street from Central Park - the busy city minutes away but sleeping in quiet at night! -PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUEST FOR DATES CALENDAR IS UP TO DATE ALL AIRBNB RESERVATIONS WILL BE HONORED Nice, comfortable, and clean private guest room with shared bathroom (2 people max) - full size bed. In very nice apartment on central Park North 4th floor walk-up. same place as Chez chic #2, max capacity of the rooms 2 people). You will share the apt with me and my little family. Daily cleaning in common areas. Located one block from Subway 2/3,B/C on 110th street, Bus M1,2,3,4 at the corner, central park across the street.  Your room: full size bed (sleeps two), desk, Digital Tv/DVD, wifi internet, A/C, closet and desk. Sheets/Towels provided. Iron/air dryer provided. Separate 

In [38]:
nylisting.totaldes[0]

"Step into our artistic spacious apartment and enjoy your artistic Guest room with original artwork from NY artists. Shared with my little family however we often out and you won't see us much. across the street from Central Park - the busy city minutes away but sleeping in quiet at night! -PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUEST FOR DATES CALENDAR IS UP TO DATE ALL AIRBNB RESERVATIONS WILL BE HONORED Nice, comfortable, and clean private guest room with shared bathroom (2 people max) - full size bed. In very nice apartment on central Park North 4th floor walk-up. same place as Chez chic #2, max capacity of the rooms 2 people). You will share the apt with me and my little family. Daily cleaning in common areas. Located one block from Subway 2/3,B/C on 110th street, Bus M1,2,3,4 at the corner, central park across the street.  Your room: full size bed (sleeps two), desk, Digital Tv/DVD, wifi internet, A/C, closet and desk. Sheets/Towels provided. Iron/air dryer provided. Separate 

## Preprocessing for NLP Modelling

In [58]:
nystring = nylisting.copy()

In [68]:
nystring.totaldes = nystring.totaldes.astype(str)

In [70]:
#tokenize the words
nystring.totaldes = nystring.totaldes.map(lambda x: nltk.tokenize.word_tokenize(x))

In [77]:
#import stopwords corpus
from nltk.corpus import stopwords
wordfilter = stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [79]:
#filtering the stop words out

def stopfilter(wordlist):
    return [word for word in wordlist if word.lower() not in wordfilter]

In [81]:
nystring.totaldes = nystring.totaldes.map(lambda x: stopfilter(x))

In [85]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()



In [86]:
#writing a function for stemming
def stemtext(wordlist):
    return ' '.join([stemmer.stem(word) for word in wordlist])



In [98]:
#stemming the string
nystring.totaldes = nystring.totaldes.map(lambda x: stemtext(x))

In [99]:
nystring.totaldes

0        step artist spaciou apart enjoy artist guest r...
1        renov apt home elev build . spaciou , renov , ...
2        find romant getaway beauti , spaciou skylit st...
3        spaciou , clean , furnish master bedroom priva...
4        welcom intern urban commun spaciou 1 bedroom p...
5        urban retreat : enjoy 500 s.f . floor 1899 bro...
6        heart ! messag avail . quiet , cool , quirki ,...
7        'm perfect locat manhattan , midtown west 'hel...
8        hello everyon thank visit bliss art space ! th...
9        welcom stay love 2 br +babi room duplex south ...
10       quiet , small one bedroom apart murray hill . ...
11       use one larg , furnish , privat room two-bedro...
12       best guest seek safe , clean , spare room fami...
13       live like new yorker instead stuffi , tini , o...
14       beauti 1br upper west side . apart great locat...
15       book three four week month juli august . pleas...
16       apart minut broadway theater , time squar , mo.

In [100]:
nystring

,id,listing_url,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,...,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,totaldes
0,2515,https://www.airbnb.com/rooms/2515,2018-03-04,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...,none,NaN,Please no cooking at night but you can warm up...,...,9.0,f,f,f,strict,f,f,3.0,1.43,step artist spaciou apart enjoy artist guest r...
1,2539,https://www.airbnb.com/rooms/2539,2018-03-04,Clean & quiet apt home by the park,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...,none,Close to Prospect Park and Historic Ditmas Park,If this room is unavailable on your desired da...,...,10.0,f,f,f,moderate,f,f,7.0,0.26,"renov apt home elev build . spaciou , renov , ..."
2,2595,https://www.airbnb.com/rooms/2595,2018-03-04,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,NaN,...,9.0,f,f,f,strict,t,t,2.0,0.29,"find romant getaway beauti , spaciou skylit st..."
3,3330,https://www.airbnb.com/rooms/3330,2018-03-04,++ Brooklyn Penthouse Guestroom ++,"This is a spacious, clean, furnished master be...","Room Features: - clean, hardwood floors - 2 la...","This is a spacious, clean, furnished master be...",none,Location is GREAT!! Right off the L train in t...,We would very much appreciate it if you give u...,...,10.0,f,f,f,strict,f,f,3.0,0.27,"spaciou , clean , furnish master bedroom priva..."
4,3647,https://www.airbnb.com/rooms/3647,2018-03-04,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,NaN,...,NaN,f,f,f,strict,t,t,1.0,NaN,welcom intern urban commun spaciou 1 bedroom p...
5,3831,https://www.airbnb.com/rooms/3831,2018-03-04,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,"My husband, Tivon, is NYC-licensed For-Hire-Ve...",...,9.0,f,f,f,moderate,f,f,1.0,4.34,urban retreat : enjoy 500 s.f . floor 1899 bro...
6,4989,https://www.airbnb.com/rooms/4989,2018-03-04,Available! Perfect location! Entire apartment!,In the heart of it all! Message me about avai...,"Quiet, cool, quirky, 1 bedroom apt. in Hell's ...",In the heart of it all! Message me about avai...,none,"Hell's Kitchen is a safe, popular, and desirab...",Enjoy!,...,9.0,f,f,f,strict,f,f,1.0,0.26,"heart ! messag avail . quiet , cool , quirki ,..."
7,5054,https://www.airbnb.com/rooms/5054,2018-03-06,Midtown West,NaN,"I'm in a perfect location of Manhattan, being ...","I'm in a perfect location of Manhattan, being ...",none,NaN,NaN,...,9.0,f,f,f,strict,f,f,1.0,0.53,"'m perfect locat manhattan , midtown west 'hel..."
8,5121,https://www.airbnb.com/rooms/5121,2018-03-04,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,none,NaN,NaN,...,9.0,f,f,f,strict,f,f,2.0,0.48,hello everyon thank visit bliss art space ! th...
9,5136,https://www.airbnb.com/rooms/5136,2018-03-05,Spacious Brooklyn Duplex + Garden,We welcome you to stay in our lovely 2 br +bab...,Sleeps 4 + baby comfortably with 6 people max ...,We welcome you to stay in our lovely 2 br +bab...,none,NaN,We have two small children- so there are many ...,...,10.0,f,f,f,strict,f,f,1.0,0.02,welcom stay love 2 br +babi room duplex south ...


In [101]:
#selecting the columns needed
wantedlist = ['id', 'totaldes']
nydoc = nystring[wantedlist]

In [202]:
nydoc.totaldes = nydoc.totaldes.map(lambda x: re.sub(r'[\/><,.()"]', '', x))
nydoc.totaldes[0]

"step artist spaciou apart enjoy artist guest room origin artwork NY artist  share littl famili howev often wo n't see us much  across street central park - busi citi minut away sleep quiet night ! -pleas book directli  need send request date calendar date airbnb reserv honor nice  comfort  clean privat guest room share bathroom  2 peopl max  - full size bed  nice apart central park north 4th floor walk-up  place chez chic # 2  max capac room 2 peopl   share apt littl famili  daili clean common area  locat one block subway 23  bc 110th street  bu m1234 corner  central park across street  room : full size bed  sleep two   desk  digit tvdvd  wifi internet  ac  closet desk  sheetstowel provid  ironair dryer provid  separ F cabl TV  internet  wifi  air condit  kitchen  buzzerwireless intercom  heat  familykid friendli  hanger  hair dryer  iron  laptop friendli workspac  children ’ book toy  window guard  pack ’ n playtravel crib"

## Build a doc2vec model using gensim

In [203]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
import multiprocessing


In [204]:
# MyDocs reading from a data frame

class MyDocs(object):
    def __iter__(self):
        for i in range(nydoc.shape[0]):
            yield TaggedDocument(words=simple_preprocess(nydoc.iloc[i,1]), tags=['%s' % nydoc.iloc[i,0]])

            


In [205]:
%%time

cores = multiprocessing.cpu_count()

if not os.path.exists('./models/listingdoc2vec2.model'):
    print("start traing doc2vec model...")
    documents = MyDocs()
    doc2vec_model = Doc2Vec(dm=1, dbow_words=1, vector_size=200, window=8, min_count=20, workers=cores)
    doc2vec_model.build_vocab(documents)
    doc2vec_model.train(documents, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    if not os.path.exists('models'):
        os.makedirs('models')
        doc2vec_model.save('./models/listingdoc2vec2.model')
    else:
        doc2vec_model.save('./models/listingdoc2vec2.model')
else:
    doc2vec_model = Doc2Vec.load('./models/listingdoc2vec2.model')





CPU times: user 316 ms, sys: 78.7 ms, total: 394 ms
Wall time: 419 ms


In [206]:
def search(keyword):
    result = []
    for name in nydoc.totaldes:
        if keyword in name.lower():
            result.append(name)
    return set(result)




In [207]:
search('new york')




{"well appoint art centric apt  incompat view  call new york home privat 1br  full bed 2  closet  wifi  cabl  like youll need corner fame central park experi new york way 's suppos  live central park  short walk away museum mile  easi access public transport allow quick west side & easi side access  privat bedroom  gloriou balconi complimentari sun greet morn privat bedroom larg privat closet  part time photo guid  involv experi  suggest  simpl provid set key kingdom apt great public transport  2-3block 6  2  & 3 train line allow easi access east & west side manhattan  also bu stop right outsid doorstep TV  cabl TV  internet  wifi  air condit  wheelchair access  kitchen  pet allow  doorman  elev  heat  washer  dryer  smoke detector  carbon monoxid detector  first aid kit  fire extinguish  essenti  shampoo  translat miss : enhosting_amenity_49  self check-in  smart lock",
 'sleek  stylish spaciou loft tribeca new york citi ’ premier downtown neighborhood  privat key elev open eclect art

In [208]:
print(doc2vec_model.docvecs.most_similar('new york', topn=20))




TypeError: '<' not supported between instances of 'str' and 'int'